# 1. Importing starting code :

In [5]:
# imports 
import numpy as np
import matplotlib.pyplot as plt
import os

# For zoomable plots in Jupyter notebooks
%matplotlib widget

# 2. Load and Process Raw Time Series Data

In [ ]:
# Load the main data CSV (mouse positions and timestamps)
data_file = '001MoDe_R1.csv'
marker_file = '001MoDe_R1.marker.csv'

# Read the header information from the data file
with open(data_file, 'r') as f:
    lines = f.readlines()
    
# Find where the actual data starts (after the empty line and column headers)
header_line = 0
for i, line in enumerate(lines):
    if line.strip() == '':
        header_line = i + 2  # Skip empty line and column header
        break

# Extract parameters from first line
params = {}
param_line = lines[0].strip()
for param in param_line.split(';'):
    if '=' in param:
        key, val = param.split('=', 1)
        params[key] = val

print("Task Parameters:")
print(f"  Center: ({params.get('centerX', 'N/A')}, {params.get('centerY', 'N/A')})")
print(f"  External Radius: {params.get('externalRadius', 'N/A')}")
print(f"  Internal Radius: {params.get('internalRadius', 'N/A')}")
print(f"  Cursor Radius: {params.get('cursorRadius', 'N/A')}")
print(f"  Task Radius: {params.get('taskRadius', 'N/A')}")
print(f"  Task Tolerance: {params.get('taskTolerance', 'N/A')}")
print(f"  Cycle Max Number: {params.get('cycleMaxNumber', 'N/A')}")
print(f"  Cycle Duration: {params.get('cycleDuration', 'N/A')} seconds")

In [ ]:
# Load the raw time series data
raw_data = np.loadtxt(data_file, delimiter=',', skiprows=header_line, dtype=str)

# Convert to appropriate types
timestamps = raw_data[:, 0].astype(np.int64)  # milliseconds since epoch
mouse_x = raw_data[:, 1].astype(np.float64)   # x position
mouse_y = raw_data[:, 2].astype(np.float64)   # y position
in_target = raw_data[:, 3].astype(np.int32)   # 1 if in target, 0 otherwise

# Convert timestamps to relative time in seconds
t0 = timestamps[0]
time_seconds = (timestamps - t0) / 1000.0

print(f"\nRaw Time Series Data Loaded:")
print(f"  Total data points: {len(timestamps)}")
print(f"  Duration: {time_seconds[-1]:.2f} seconds ({time_seconds[-1]/60:.2f} minutes)")
print(f"  Sampling rate: ~{len(timestamps)/time_seconds[-1]:.1f} Hz")
print(f"  First timestamp: {timestamps[0]}")
print(f"  Last timestamp: {timestamps[-1]}")

In [ ]:
# Load marker events from the marker CSV file
with open(marker_file, 'r') as f:
    marker_lines = f.readlines()

# Find where marker data starts
marker_start = 0
for i, line in enumerate(marker_lines):
    if line.strip() == '':
        marker_start = i + 1
        break

# Parse marker events
marker_events = []
for line in marker_lines[marker_start:]:
    if line.strip() and not line.strip().startswith('Var'):
        parts = line.strip().split(',')
        if len(parts) >= 2:
            marker_events.append({
                'datetime': parts[0],
                'timestamp': int(parts[1]) if parts[1].isdigit() else None,
                'event': parts[2] if len(parts) > 2 else ''
            })

print(f"\nMarker Events Loaded:")
print(f"  Total events: {len(marker_events)}")
print("\nEvent Timeline:")
for event in marker_events:
    if event['timestamp']:
        rel_time = (event['timestamp'] - t0) / 1000.0
        print(f"  {rel_time:6.1f}s: {event['event']}")

# 3. Visualize Raw Time Series Data

In [ ]:
# Plot the raw time series data
fig, axes = plt.subplots(3, 1, figsize=(12, 8), sharex=True)

# Plot X position over time
axes[0].plot(time_seconds, mouse_x, 'b-', linewidth=0.5, alpha=0.7)
axes[0].set_ylabel('X Position (pixels)')
axes[0].set_title('Mouse X Position Over Time')
axes[0].grid(True, alpha=0.3)

# Plot Y position over time
axes[1].plot(time_seconds, mouse_y, 'r-', linewidth=0.5, alpha=0.7)
axes[1].set_ylabel('Y Position (pixels)')
axes[1].set_title('Mouse Y Position Over Time')
axes[1].grid(True, alpha=0.3)

# Plot target status over time
axes[2].fill_between(time_seconds, 0, in_target, alpha=0.5, color='green', label='In Target')
axes[2].set_ylabel('In Target')
axes[2].set_xlabel('Time (seconds)')
axes[2].set_title('Target Hit Status Over Time')
axes[2].set_ylim([-0.1, 1.1])
axes[2].set_yticks([0, 1])
axes[2].set_yticklabels(['Out', 'In'])
axes[2].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("\nRaw time series plots generated successfully!")

In [ ]:
# Plot the circular trajectory (2D visualization)
fig, ax = plt.subplots(figsize=(8, 8))

# Color code by time
scatter = ax.scatter(mouse_x, mouse_y, c=time_seconds, s=1, cmap='viridis', alpha=0.5)

# Add task parameters
center_x = int(params.get('centerX', 552))
center_y = int(params.get('centerY', 330))
external_r = int(params.get('externalRadius', 250))
internal_r = int(params.get('internalRadius', 170))

# Draw the circular target region
circle_outer = plt.Circle((center_x, center_y), external_r, color='blue', fill=False, linewidth=2, label='Outer boundary')
circle_inner = plt.Circle((center_x, center_y), internal_r, color='red', fill=False, linewidth=2, label='Inner boundary')
ax.add_patch(circle_outer)
ax.add_patch(circle_inner)

# Mark the center
ax.plot(center_x, center_y, 'k+', markersize=15, label='Center')

ax.set_xlabel('X Position (pixels)')
ax.set_ylabel('Y Position (pixels)')
ax.set_title('Mouse Trajectory in Circular Task')
ax.set_aspect('equal')
ax.legend()
ax.grid(True, alpha=0.3)

# Add colorbar for time
cbar = plt.colorbar(scatter, ax=ax)
cbar.set_label('Time (seconds)')

plt.tight_layout()
plt.show()

print("\nCircular trajectory plot generated successfully!")